In [24]:
# import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#Importo las funciones propias de pipelines
from pipelines import *

# *** GRÁFICO DEL APARTDO 2 ***

# Ampliar el ancho de la página
# st.set_page_config(layout="wide")
df_customers = pd.read_csv('./../datasets_limpios/olist_order_customer.csv')
df_orders = pd.read_csv('./../datasets_limpios/olist_orders_dataset.csv')

# Título de sección
# st.header("2. Estados con mayor porcentaje de pedidos")

# Relacionar los datasets por 'customer_id'
df_merged = pd.merge(df_customers, df_orders, on='customer_id', how='inner')

# Asegurar mayúsculas en estados y formato título en ciudades
df_merged['customer_state'] = df_merged['customer_state'].astype(str).str.upper()
df_merged['customer_city'] = df_merged['customer_city'].astype(str).str.title()

# Calcular número de clientes únicos por ciudad
clientes_por_ciudad = df_merged.groupby(['customer_state', 'customer_city'])['customer_unique_id'].nunique().reset_index()
clientes_por_ciudad.columns = ['Estado', 'Ciudad', 'Número de clientes']

# Calcular número de pedidos por ciudad (conteo de order_id por ciudad)
pedidos_por_ciudad = df_merged.groupby(['customer_state', 'customer_city'])['order_id'].count().reset_index()
pedidos_por_ciudad.columns = ['Estado', 'Ciudad', 'Número de pedidos']

# Combinar ambas tablas
tabla = pedidos_por_ciudad.merge(clientes_por_ciudad, on=['Estado', 'Ciudad'])

# Calcular ratio como número entero
tabla['Ratio de Pedidos por Cliente'] = (
    tabla['Número de pedidos'] / tabla['Número de clientes']
).round(2)

# Calcular número de pedidos por cliente
pedidos_por_cliente = (
    df_merged.groupby('customer_unique_id')['order_id']
    .count()
    .reset_index()
    .rename(columns={'order_id': 'Número de pedidos'})
)

# Ordenar de mayor a menor
pedidos_por_cliente = pedidos_por_cliente.sort_values(by='Número de pedidos', ascending=False)

# Mostrar los clientes con más pedidos (por ejemplo, los 10 primeros)
print(pedidos_por_cliente.head(5))
print(tabla.sort_values(by='Número de pedidos', ascending=False).head(5))



# Select de estado
# estados_unicos = sorted(tabla['Estado'].unique())
# estado_seleccionado = st.selectbox("Selecciona un estado para ver sus ciudades", estados_unicos)

# Filtrar por estado y preparar Top 10
# datos_estado = tabla[tabla['Estado'] == estado_seleccionado].copy()
# datos_estado = datos_estado.sort_values(by='Número de pedidos', ascending=False)

# top10_estados = datos_estado.head(10).copy()
# total_top10 = top10_estados['Número de pedidos'].sum()

# # Calcular porcentaje dentro del top 10
# top10_estados['Porcentaje'] = (
#     top10_estados['Número de pedidos'] / total_top10 * 100
# ).round(2)

# # Gráfico circular
# fig, ax = plt.subplots(figsize=(8, 8))
# colors = plt.cm.PuBu(np.linspace(0.3, 1, len(top10_estados)))

# ax.pie(
#     top10_estados['Porcentaje'],
#     labels=top10_estados['Ciudad'],
#     autopct='%1.1f%%',
#     startangle=140,
#     colors=colors,
#     textprops={'fontsize': 9},
#     wedgeprops=dict(edgecolor='white'),
#     labeldistance=1.05
# )

# ax.set_title(f'Top 10 ciudades por pedidos en {estado_seleccionado}', fontsize=14)
# ax.axis('equal')

# # Mostrar gráfico y tabla
# col1, col2 = st.columns([1.3, 1.7])

# with col1:
#     st.pyplot(fig)

# with col2:
#     st.subheader(f"Detalle de ciudades en {estado_seleccionado}")
#     tabla_filtrada = top10_estados[
#         ['Ciudad', 'Número de pedidos', 'Número de clientes', 'Porcentaje', 'Ratio de Pedidos por Cliente']
#     ].reset_index(drop=True)

#     st.dataframe(tabla_filtrada, use_container_width=True)

                     customer_unique_id  Número de pedidos
52973  8d50f5eadf50201ccdcedfb9e2ac8455                 17
23472  3e43e6105506432c953e165fb2acf44c                  9
10354  1b6c7548a2a1f9037c1fd3ddfed95f33                  7
37797  6469f99c1f9dfae7733b25662e7f1782                  7
76082  ca77025e7201e3b30c44b472ff346268                  7
     Estado          Ciudad  Número de pedidos  Número de clientes  \
4176     SP       Sao Paulo              15540               14984   
2788     RJ  Rio De Janeiro               6882                6620   
1062     MG  Belo Horizonte               2773                2672   
601      DF        Brasilia               2131                2069   
2406     PR        Curitiba               1521                1465   

      Ratio de Pedidos por Cliente  
4176                          1.04  
2788                          1.04  
1062                          1.04  
601                           1.03  
2406                          1.04  


In [25]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# 1. Cargar tu dataset y agrupar
df = pd.read_csv('./../datasets_limpios/olist_order_customer.csv')
clientes_por_estado = (
    df.groupby('customer_state')['customer_unique_id']
      .nunique()
      .reset_index()
      .rename(columns={'customer_state':'Estado',
                       'customer_unique_id':'Número de clientes'})
)
# Llevar códigos a mayúsculas
clientes_por_estado['Estado'] = clientes_por_estado['Estado'].str.upper()

# 2. Cargar GeoJSON y mapear nombre -> sigla
url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'
br_estados = gpd.read_file(url)

estado_siglas = {
    'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM',
    'Bahia': 'BA', 'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES',
    'Goiás': 'GO', 'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG', 'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR',
    'Pernambuco': 'PE', 'Piauí': 'PI', 'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS', 'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC',
    'São Paulo': 'SP', 'Sergipe': 'SE', 'Tocantins': 'TO'
}
br_estados['sigla'] = br_estados['name'].map(estado_siglas)

# Llevar siglas a mayúsculas (ya lo están) o a minúsculas con .str.lower()
# Aquí dejamos ambas en mayúsculas
# br_estados['sigla'] = br_estados['sigla'].str.upper()

# 3. Hacer el merge corregido
gdf = br_estados.merge(
    clientes_por_estado,
    left_on='sigla',
    right_on='Estado',
    how='inner'
)

# 4. Comprobar que hay geometrías
assert not gdf.empty, "El merge siguió vacío: revisa tus códigos de estado"
assert gdf['geometry'].notna().all(), "Algunas geometrías son nulas"

# 5. Graficar forzando aspecto igual
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(
    column='Número de clientes',
    cmap='OrRd',
    linewidth=0.8,
    edgecolor='0.8',
    legend=True,
    ax=ax,
    aspect='equal'   # fuerza el aspect antes de hacer cálculos problemáticos
)
ax.set_title('Número de clientes por estado en Brasil', fontsize=16)
ax.axis('off')
# Después de configurar tu gráfico…
fig.savefig('clientes_por_estado_brasil.png', dpi=300, bbox_inches='tight')
print("Mapa guardado en clientes_por_estado_brasil.png")


Mapa guardado en clientes_por_estado_brasil.png


In [29]:
# Ver si hay atípicos en los días de retraso

import matplotlib.pyplot as plt
import seaborn as sns
df_customers = pd.read_csv('./../datasets_limpios/olist_order_customer.csv')
df_orders = pd.read_csv('./../datasets_limpios/olist_orders_dataset.csv')
df_sellers = pd.read_csv('./../datasets_limpios/olist_sellers_dataset.csv')
df_reviews = pd.read_csv('./../datasets_limpios/olist_order_reviews_dataset_clean.csv')
df_order_items = pd.read_csv('./../datasets_limpios/olist_order_items_dataset.csv')


df_customers_reviews = pd.merge(
    pd.merge(df_reviews, df_orders, on='order_id', how='inner'),
    df_customers, on='customer_id', how='inner'
)

datetime_params = {
        "review_creation_date": "%Y-%m-%dT%H:%M:%S.%f",
        "review_answer_timestamp": "%Y-%m-%dT%H:%M:%S.%f",
        "order_purchase_timestamp": "%Y-%m-%d %H:%M:%S",
        "order_approved_at": "%Y-%m-%d %H:%M:%S",
        "order_delivered_carrier_date": "%Y-%m-%d %H:%M:%S",
        "order_delivered_customer_date": "%Y-%m-%d %H:%M:%S",
        "order_estimated_delivery_date": "%Y-%m-%d",
    }

df_customers_reviews = clean_datetime_columns_pandas(df_customers_reviews, datetime_params)


df_customers_reviews['dias_retraso'] = (
    df_customers_reviews['order_delivered_customer_date'].dt.floor('D') - df_customers_reviews['order_estimated_delivery_date']
).dt.days.astype('Int64')



# Filtrar solo días de retraso positivos
dias_retraso_positivos = df_customers_reviews[df_customers_reviews['dias_retraso'] > 0]['dias_retraso']

# Estadísticas básicas
print(dias_retraso_positivos.describe())

# Boxplot para visualizar outliers
plt.figure(figsize=(8,4))
sns.boxplot(x=dias_retraso_positivos)
plt.title('Boxplot de días de retraso positivos')
plt.show()

# Método IQR para detectar outliers
Q1 = dias_retraso_positivos.quantile(0.25)
Q3 = dias_retraso_positivos.quantile(0.75)
IQR = Q3 - Q1

outliers = dias_retraso_positivos[
    (dias_retraso_positivos < Q1 - 1.5 * IQR) | 
    (dias_retraso_positivos > Q3 + 1.5 * IQR)
]

print(f"Número de outliers detectados: {len(outliers)}")
print(outliers.describe())

# Percentiles para entender valores extremos
print(dias_retraso_positivos.quantile([0.01, 0.05, 0.95, 0.99]))

plt.figure(figsize=(10, 5))
sns.boxplot(x=dias_retraso_positivos)

# Cuartiles
Q1 = dias_retraso_positivos.quantile(0.25)
Q3 = dias_retraso_positivos.quantile(0.75)
IQR = Q3 - Q1

# Definición outliers fuertes (> 3*IQR)
outliers_fuertes = dias_retraso_positivos[
    (dias_retraso_positivos < Q1 - 3 * IQR) | 
    (dias_retraso_positivos > Q3 + 3 * IQR)
]

print(f"Número de outliers fuertes detectados: {len(outliers_fuertes)}")
print(outliers_fuertes.describe())


# Marcar outliers fuertes en rojo grande para destacarlos
for val in outliers_fuertes.values:
    plt.scatter(val, 0, color='red', s=100, edgecolor='black', zorder=10)

plt.title('Boxplot de días de retraso positivos con outliers fuertes destacados')
plt.xlabel('Días de retraso')
plt.show()

plt.figure(figsize=(10, 5))
sns.boxplot(x=dias_retraso_positivos)

for val in outliers_fuertes.values:
    plt.scatter(val, 0, color='red', s=100, edgecolor='black', zorder=10)

plt.title('Boxplot de días de retraso positivos con outliers fuertes destacados')
plt.xlabel('Días de retraso')

plt.savefig("boxplot_dias_retraso_outliers_fuertes.png")


*** Columna 'review_creation_date': convertida con formato %Y-%m-%dT%H:%M:%S.%f ***
*** Columna 'review_answer_timestamp': convertida con formato %Y-%m-%dT%H:%M:%S.%f ***
*** Columna 'order_purchase_timestamp': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_approved_at': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_delivered_carrier_date': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_delivered_customer_date': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_estimated_delivery_date': convertida con formato %Y-%m-%d ***
count       6410.0
mean     10.526833
std      14.445316
min            1.0
25%            3.0
50%            7.0
75%           13.0
max          188.0
Name: dias_retraso, dtype: Float64
Número de outliers detectados: 382
count        382.0
mean      51.91623
std      31.936425
min           29.0
25%           33.0
50%           40.0
75%           54.0
max          188.0
Name: dias_retraso, dtype: Float

C:\Users\sfernandeznevado\AppData\Local\Temp\ipykernel_9804\177213852.py:46: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\sfernandeznevado\AppData\Local\Temp\ipykernel_9804\177213852.py:88: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [30]:
# Filtrar DataFrame solo con outliers fuertes
df_outliers_fuertes = df_customers_reviews.loc[outliers.index]

# Detectar casos donde la fecha estimada es anterior a la compra o a la aprobación
cond_estimada_antes_compra = df_outliers_fuertes['order_estimated_delivery_date'] < df_outliers_fuertes['order_purchase_timestamp'].dt.floor('D')
cond_estimada_antes_aprobacion = df_outliers_fuertes['order_estimated_delivery_date'] < df_outliers_fuertes['order_approved_at'].dt.floor('D')

# Pedidos con problemas en fechas estimadas
problemas_fechas = df_outliers_fuertes[cond_estimada_antes_compra | cond_estimada_antes_aprobacion]

print(f"Número de outliers fuertes con fecha estimada anterior a compra o aprobación: {len(problemas_fechas)}")

# Mostrar algunos ejemplos
print(problemas_fechas[['order_id', 'order_purchase_timestamp', 'order_approved_at', 'order_estimated_delivery_date', 'dias_retraso']].head())

# Opcional: ver porcentaje respecto al total de outliers fuertes
porcentaje_problemas = len(problemas_fechas) / len(df_outliers_fuertes) * 100
print(f"Porcentaje de outliers fuertes con fechas estimadas erróneas: {porcentaje_problemas:.2f}%")


Número de outliers fuertes con fecha estimada anterior a compra o aprobación: 0
Empty DataFrame
Columns: [order_id, order_purchase_timestamp, order_approved_at, order_estimated_delivery_date, dias_retraso]
Index: []
Porcentaje de outliers fuertes con fechas estimadas erróneas: 0.00%


In [ ]:

# Conversiones
df_customers_reviews['order_purchase_timestamp'] = pd.to_datetime(df_customers_reviews['order_purchase_timestamp'])
df_customers_reviews['order_approved_at'] = pd.to_datetime(df_customers_reviews['order_approved_at'])
df_customers_reviews['order_estimated_delivery_date'] = pd.to_datetime(df_customers_reviews['order_estimated_delivery_date']).dt.floor('D')

# Fechas solo día
df_customers_reviews['order_purchase_date'] = df_customers_reviews['order_purchase_timestamp'].dt.floor('D')
df_customers_reviews['order_approved_date'] = df_customers_reviews['order_approved_at'].dt.floor('D')

# Fecha mínima entre compra y aprobación, ignorando nulos
df_customers_reviews['fecha_min_compra_aprob'] = df_customers_reviews[['order_purchase_date', 'order_approved_date']].min(axis=1)

# Máscara para filas con fecha estimada y fecha mínima no nulas
mask = df_customers_reviews['order_estimated_delivery_date'].notna() & df_customers_reviews['fecha_min_compra_aprob'].notna()

# Detectar fallos: fecha estimada anterior a la fecha mínima de compra/aprobación
fallo_estimado_menor = pd.Series(False, index=df_customers_reviews.index)
fallo_estimado_menor.loc[mask] = df_customers_reviews.loc[mask, 'order_estimated_delivery_date'] < df_customers_reviews.loc[mask, 'fecha_min_compra_aprob']

# Filtrar registros con fallo
registros_fallos = df_customers_reviews[fallo_estimado_menor].copy()

print(f"Número de registros con fecha estimada anterior a compra o aprobación: {len(registros_fallos)}")

# Añadir columna para fallo
registros_fallos['fallo_estimado_menor'] = True

# Mostrar resultado
print(registros_fallos[[
    'order_id',
    'order_purchase_timestamp',
    'order_approved_at',
    'order_estimated_delivery_date',
    'fecha_min_compra_aprob',
    'fallo_estimado_menor'
]])


Número de registros con fecha estimada anterior a compra o aprobación: 0
Empty DataFrame
Columns: [order_id, order_purchase_timestamp, order_approved_at, order_estimated_delivery_date, fecha_min_compra_aprob, fallo_estimado_menor]
Index: []


In [ ]:
# df_customers = pd.read_csv('./../datasets_limpios/olist_order_customer.csv')
# df_orders = pd.read_csv('./../datasets_limpios/olist_orders_dataset.csv')
# df_sellers = pd.read_csv('./../datasets_limpios/olist_sellers_dataset.csv')
# df_reviews = pd.read_csv('./../datasets_limpios/olist_order_reviews_dataset_clean.csv')
# df_order_items = pd.read_csv('./../datasets_limpios/olist_order_items_dataset.csv')
# df_order_items = pd.read_csv('./../datasets_limpios/olist_order_items_dataset.csv')
df_product = pd.read_csv('./../datasets_limpios/olist_products_dataset.csv')

print(df_order_items.info())
print(df_product.info())

# Productos únicos que aparecen en los pedidos
productos_en_pedidos = set(df_order_items['product_id'].unique())

# Todos los productos del catálogo
productos_catalogo = set(df_product['product_id'].unique())

# Productos que están en el catálogo pero no en pedidos
productos_sin_pedido = productos_catalogo - productos_en_pedidos

print(f"Número de productos sin pedidos: {len(productos_sin_pedido)}")




df_product['product_category_name'].unique()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32951 non-null  object 
 2   product_name_lenght    

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
